In [ ]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import json
import sqlite3
from urllib.parse import urlparse, parse_qs

class BookServer(BaseHTTPRequestHandler):

    def _connect_db(self):
        return sqlite3.connect('books.db')

    def _send_response(self, status_code, content):
        self.send_response(status_code)
        self.send_header('Content-type', 'application/json')
        self.end_headers()
        self.wfile.write(content.encode())

    def do_GET(self):
        parsed_path = urlparse(self.path)
        path = parsed_path.path
        
        if path.startswith('/books'):
            book_id = path.split('/')[-1]
            if book_id.isdigit():
                conn = self._connect_db()
                cursor = conn.cursor()
                cursor.execute('SELECT * FROM books WHERE id = ?', (book_id,))
                book = cursor.fetchone()
                conn.close()
                
                if book:
                    response = json.dumps({
                        'id': book[0],
                        'book_name': book[1],
                        'author': book[2],
                        'publisher': book[3]
                    })
                    self._send_response(200, response)
                else:
                    self._send_response(404, json.dumps({'error': 'Book not found'}))
            else:
                conn = self._connect_db()
                cursor = conn.cursor()
                cursor.execute('SELECT * FROM books')
                books = cursor.fetchall()
                conn.close()
                
                response = json.dumps([{
                    'id': book[0],
                    'book_name': book[1],
                    'author': book[2],
                    'publisher': book[3]
                } for book in books])
                self._send_response(200, response)
                
    def do_POST(self):
        if self.path == '/books':
            content_length = int(self.headers['Content-Length'])
            post_data = self.rfile.read(content_length).decode()
            data = json.loads(post_data)
            
            conn = self._connect_db()
            cursor = conn.cursor()
            cursor.execute('INSERT INTO books (book_name, author, publisher) VALUES (?, ?, ?)',
                           (data['book_name'], data['author'], data['publisher']))
            conn.commit()
            conn.close()
            
            self._send_response(201, json.dumps({'message': 'Book created'}))

    def do_PUT(self):
        if self.path.startswith('/books/'):
            book_id = self.path.split('/')[-1]
            if book_id.isdigit():
                content_length = int(self.headers['Content-Length'])
                put_data = self.rfile.read(content_length).decode()
                data = json.loads(put_data)
                
                conn = self._connect_db()
                cursor = conn.cursor()
                cursor.execute('UPDATE books SET book_name = ?, author = ?, publisher = ? WHERE id = ?',
                               (data['book_name'], data['author'], data['publisher'], book_id))
                conn.commit()
                
                if cursor.rowcount > 0:
                    response = json.dumps({'message': 'Book updated'})
                else:
                    response = json.dumps({'error': 'Book not found'})
                conn.close()
                
                self._send_response(200, response)
            else:
                self._send_response(400, json.dumps({'error': 'Invalid ID'}))

    def do_DELETE(self):
        if self.path.startswith('/books/'):
            book_id = self.path.split('/')[-1]
            if book_id.isdigit():
                conn = self._connect_db()
                cursor = conn.cursor()
                cursor.execute('DELETE FROM books WHERE id = ?', (book_id,))
                conn.commit()
                
                if cursor.rowcount > 0:
                    response = json.dumps({'message': 'Book deleted'})
                else:
                    response = json.dumps({'error': 'Book not found'})
                conn.close()
                
                self._send_response(200, response)
            else:
                self._send_response(400, json.dumps({'error': 'Invalid ID'}))

def run(server_class=HTTPServer, handler_class=BookServer, port=8080):
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)
    print(f'Starting httpd on port {port}...')
    httpd.serve_forever()

if __name__ == '__main__':
    run()
